# HyDroForM Use Case

This notebook showcases how to use the HyDroForM use case, which integrates hydrological modeling with machine learning techniques for efficient and accurate flood forecasting.

The use case setup utilizes openEO to trigger container processing on OSCAR.

## Table of Contents

TODO

## Importing Required Libraries

In [ ]:
import openeo
from openeo.local import LocalConnection
localconn = LocalConnection("./")

## Authenticate with openEO backend

EGI Check-in allows us to reuse the same credentials for OSCAR

In [ ]:
connection = connection = openeo.connect("https://openeo.intertwin.fedcloud.eu/1.1.0/").authenticate_oidc()

## Setting up the workflow

The `run_oscar` openEO process developed in InterTwin is used to trigger the execution of the workflow on OSCAR.

It allows us to set up the workflow by providing the service definitions, OSCAR backend URL and the input parameters.

As a best practice for the greatest compatibility with openEO, the container shall return a STAC collection as output.

This is necessary to be able to load the results back into openEO for further processing or visualization.

### HydroMT step

The HydroMT step takes user input such as `model resolution`, `date range`, `location`, specific `dataset` etc. to set up a configuration for the hydrological model ran in the next step.

In [ ]:
hydromt_step = localconn.datacube_from_process(
    "run_oscar",
    oscar_endpoint="https://oscar-ukri.intertwin.fedcloud.eu/",
    service="hydroform-hydromt",
    output="output",
    input_stac_url="https://stac.example.com/item/123",
    local_refresh_token="/home/jzvolensky/.local/share/openeo-python-client/refresh-tokens.json",
    oscar_script=None,
    service_config="./configs/oscar_hydromt_svc.yaml",
    context={
        "AWS_ACCESS_KEY_ID": "your_access_key_id",
        "AWS_SECRET_ACCESS_KEY": "your_secret_access_key",
        "RES": "0.008999999999",
        "PRECIP_FN": "emo1_stac",
        "STARTTIME": "2001-01-01T00:00:00",
        "ENDTIME": "2001-03-31T00:00:00",
    }
)

hydromt_step

### Executing a single step
in case we want to run just a single step, we can call the `execute` method at any time in the workflow setup. This will return the output as a STAC collection string.

In [ ]:
result =hydromt_step.execute()
result

### Wflow step

Wflow is Deltares’ solution for modelling hydrological processes, allowing users to account for precipitation, interception, snow accumulation and melt, evapotranspiration, soil water, surface water and groundwater recharge in a fully distributed environment.

It uses the preprocessed data from the HydroMT step to run the hydrological simulation for the specified region and time period.

In [ ]:
wflow_step = localconn.datacube_from_process(
    "run_oscar",
    oscar_endpoint="https://oscar-ukri.intertwin.fedcloud.eu/",
    service="hydroform-wflow",
    output="output2",
    input_stac_url=hydromt_step,
    local_refresh_token="/home/jzvolensky/.local/share/openeo-python-client/refresh-tokens.json",
    oscar_script=None,
    service_config="./configs/oscar_wflow_svc.yaml",
    context={
        "AWS_ACCESS_KEY_ID": "your_access_key_id",
        "AWS_SECRET_ACCESS_KEY": "your_secret_access_key"
    }
)

In [ ]:
wflow_result = wflow_step.execute()

### Surrogate training step

In [ ]:
surrogate_step = localconn.datacube_from_process(
    "run_oscar",
    oscar_endpoint="https://oscar-ukri.intertwin.fedcloud.eu/",
    service="hydroform-surrogate",
    output="output3",
    input_stac_url=wflow_step,
    local_refresh_token="/home/jzvolensky/.local/share/openeo-python-client/refresh-tokens.json",
    oscar_script=None,
    service_config="./configs/oscar_surrogate_svc.yaml",
    context={
        "AWS_ACCESS_KEY_ID": "your_access_key_id",
        "AWS_SECRET_ACCESS_KEY": "your_secret_access_key",
        "MLFLOW_TRACKING_URI": "your_mlflow_tracking_uri",
        "MLFLOW_TRACKING_USERNAME": "your_mlflow_username",
        "MLFLOW_TRACKING_PASSWORD": "your_mlflow_password",
        "MLFLOW_TRACKING_INSECURE_TLS": "true"
    }
)

surrogate_step

### Executing the training

In [ ]:
surrogate_result = surrogate_step.execute()
surrogate_result